In [ ]:
# Import packages.
import figure_utilities
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
plt.rcParams["figure.dpi"] = 150
plt.rcParams['savefig.dpi'] = 150
from scipy import stats
from sklearn.model_selection import train_test_split
import numpy as np
import os
import pandas as pd
from mixture_sampler import MixtureSampler

In [ ]:
# Define paths.
INPUT_DATA = "../data/mcas.xls"
OUTPUT_FIGURES = "../figures/"
OUTPUT_TABLES = "../tables/"

In [ ]:
# Load data.
df = pd.read_excel(INPUT_DATA)
df["constant"] = 1 # add a column of 1's
y_variable = 'totsc4'
X_variables = ['constant', 'tchratio', 'pctel', 'percap', 'lnch_pct']
k = len(X_variables)
X = df[X_variables].values
y = df[y_variable].values

In [ ]:
# Log income before including in model.
df['percap'] = np.log(df['percap'])

# 1. Summary Statistics

In [ ]:
# Table 1: Summary Statistics
summary_statistics_display_names = {'mean': "Mean",
                                    '50%': "Median",
                                    'std': "SD",
                                    'min': "Minimum", 
                                    'max': "Maximum"}
variable_display_names = {'totsc4': 'MCAS Score'}  # TODO
summary_statistics_table = (df
                            .describe()
                            .T[['mean', '50%', 'std', 'min', 'max']]
                            .drop(index='code')
                            .drop(index='totsc8')
                            .rename(columns=summary_statistics_display_names))
# Export to LaTeX.
filename = os.path.join(OUTPUT_TABLES, "summary_statistics.tex")
latex = (summary_statistics_table
         .rename(index=variable_display_names)
         .rename(columns=summary_statistics_display_names)
         .style
         .format(formatter={
                            'Mean': "{:,.2f}",
                            'Median': "{:,.2f}",
                            'S.D.': "{:,.2f}",
                            'Maximum': "{:,.2f}",
                            'Minimum': "{:,.2f}"})
         .format_index("\\textit{{{}}}", escape="latex", axis=0, level=0)
         .to_latex(None,
                   column_format="lccccc",
                   hrules=True,
                   clines="skip-last;data")).replace("{*}", "{4cm}")
with open(filename, 'w') as file:
    file.write(latex)
summary_statistics_table

# 2. Simulations

In [ ]:
# Define an np.random.Generator to use for sampling.
generator = np.random.default_rng(seed=7)
N = 1000

In [ ]:
## Set prior parameters.
prior_parameters = {}
prior_parameters['m'] = 3

# mu_j (location of mixture components) prior parameters.
prior_parameters['mu_j_underbar'] = 0
prior_parameters['h_underbar'] = 0.25  # This value will be re-drawn from the prior on h for every simulation.
prior_parameters['h_mu_underbar'] = 0.25

# beta prior parameters.
prior_parameters['beta_underbar'] = (np.linalg.inv(X.T @ X) @ (X.T @ y))
prior_parameters['H_underbar'] = X.T @ X

# h_j (scale of mixture components) prior parameters.
prior_parameters['nu_j_underbar'] = 1  # NOTE: Right now all h_j's are drawn from the same distribution.
prior_parameters['s2_j_underbar'] = 1

# h (scaling term for scale of mixture components) prior parameters.
prior_parameters['nu_underbar'] = 1
prior_parameters['s2_underbar'] = 1

# alphas prior parameters.
prior_parameters['alpha_underbar'] = np.ones(prior_parameters['m'])

# Define class which samples from priors
mixture_sampler = MixtureSampler(generator, prior_parameters, X, y)

## 2a. Prior Predictive Analysis

In this section, we perform prior predictive analysis using the following steps.

1. Draw all unobservables from their priors.
2. Sample a fictitious sample of y_i's from the likelihood function, conditional on these unobservables.
3. Calculate the mean, standard deviation, and largest deviation from the mean using this fictious sample.
4. Repeat 1000 times so that we have 1000 realizations each of mean, standard deviation, and largest deviation from the mean.
5. Calculate the mean, standard deviation, and largest deviation from the mean using the observed data.
6. For each statistic, plot a histogram of the 1000 realizations produced using the fictitious data.
7. For each statistic, overlay the realization calculated using the observed data.
8. Report the "p-value"

Note that while we explore more complex models with 5 and 7 mixture components later in our analysis, we only perform prior predictive analysis for our simplest model, with 3 mixture components.

In [ ]:
fictitious_samples_prior = []  # Will contain N vectors, each containinng 220 fictitious observations of y.
fictitious_samples_posterior = []

for draw in range(N):
    # Priors
    # Draw unobservables from their priors.
    current_draw_of_betas_prior, current_draw_of_h_prior, current_draw_of_alpha_js_prior, current_draw_of_h_js_prior, current_draw_of_mu_js_prior = mixture_sampler.sample_from_prior()

    # Draw s_is.
    s_is_prior = generator.choice(prior_parameters["m"], size=len(y), p=current_draw_of_alpha_js_prior)

    # Draw y_is
    current_means_for_each_observation_prior =  current_draw_of_mu_js_prior[s_is_prior] + (current_draw_of_betas_prior @ X.T)
    current_h_js_for_each_observation_prior = current_draw_of_h_js_prior[s_is_prior]
    current_y_is_prior = generator.multivariate_normal(np.ravel(current_means_for_each_observation_prior), np.diag(1 / current_h_js_for_each_observation_prior))
    fictitious_samples_prior.append(current_y_is_prior)
    
#     # Posteriors
#     #Draw unobservables from their posteriors.
    
#     current_draw_of_betas_posterior, current_draw_of_h_posterior, current_draw_of_alpha_js_posterior,\
#     current_draw_of_h_js_posterior, current_draw_of_mu_js_posterior, s_is_posterior = mixture_sampler.sample_from_posterior()

#     # Draw y_is
#     current_means_for_each_observation_posterior =  current_draw_of_mu_js_posterior[s_is_posterior] + (current_draw_of_betas_posterior @ X.T)
#     current_h_js_for_each_observation_posterior = current_draw_of_h_js_posterior[s_is_posterior]
#     current_y_is_posterior = generator.multivariate_normal(np.ravel(current_means_for_each_observation_posterior), np.diag(1 / current_h_js_for_each_observation_posterior))
#     fictitious_samples_posterior.append(current_y_is_posterior)

# Convert list of fictitious samples into array.
fictitious_samples_prior = np.vstack(fictitious_samples_prior)
# fictitious_samples_posterior = np.vstack(fictitious_samples_posterior)

# Calculate statistics on fictitious samples using unobservables from the prior.
fictitious_means_prior = np.mean(fictitious_samples_prior, axis = 1)
fictitious_stds_prior = np.std(fictitious_samples_prior, axis = 1)
fictitious_maxes_minus_means_prior = np.max(fictitious_samples_prior, axis = 1) - fictitious_means_prior
fictitious_statistics_prior = [fictitious_means_prior, fictitious_stds_prior, fictitious_maxes_minus_means_prior]

# # Calculate statistics on fictitious samples using unobservables from the posterior.
# fictitious_means_posterior = np.mean(fictitious_samples_posterior, axis = 1)
# fictitious_stds_posterior = np.std(fictitious_samples_posterior, axis = 1)
# fictitious_maxes_minus_means_posterior = np.max(fictitious_samples_posterior, axis = 1) - fictitious_means_posterior
# fictitious_statistics_posterior = [fictitious_means_posterior, fictitious_stds_posterior, fictitious_maxes_minus_means_posterior]

# Calculate true values of each statistic.
true_mean = np.mean(y)
true_std = np.std(y)
true_max_minus_mean = np.max(y) - true_mean

In [ ]:
# Plot.
fig, axes = plt.subplots(1, 3, figsize=(15, 7.5), sharey=False)
fig.suptitle("Prior Predictive Analysis Using Various Statistics")
titles = ["Mean", "S.D.", "Max. minus Mean"]
xlabels = ["$\\bar{y} = \\frac{1}{n}\sum_{i=1}^{n}y_i$",
          "$s_y = \sqrt{\\frac{1}{N-1} \sum_{i=1}^N (x_i - \overline{x})^2}$",
          "$\Delta_y = max\\{y_1, y_2, ..., y_n\\} - \\bar{y}$"]
vline_positions = [true_mean, true_std, true_max_minus_mean]
vline_labels = ["$\\bar{y}^0$", "$s_y^0$", "$\Delta_y^0$"]

for ax, fictitious_statistic_prior, title, xlabel, vline_position, vline_label in zip(axes, fictitious_statistics_prior, titles, xlabels, vline_positions, vline_labels):
    current_statistic_prior = pd.Series(np.reshape(fictitious_statistic_prior, -1))

# for ax, fictitious_statistic_prior, fictitious_statistic_posterior, title, xlabel, vline_position, vline_label in zip(axes, fictitious_statistics_prior, fictitious_statistics_posterior, titles, xlabels, vline_positions, vline_labels):
#     current_statistic_prior = pd.Series(np.reshape(fictitious_statistic_prior, -1))
#     current_statistic_posterior = pd.Series(np.reshape(fictitious_statistic_posterior, -1))
    
    current_statistic_prior.plot(ax = ax, kind = 'kde', color = 'blue', alpha = 0.7)
#     current_statistic_posterior.plot(ax = ax, kind = 'kde', color = 'red', alpha = 0.7)
    figure_utilities.plot_labeled_vline(ax, vline_position, vline_label)
    
    ax.set_xlabel(xlabel)
    ax.set_ylabel("")
    ax.set_title(title)
#     figure_utilities.plot_histogram(ax, fictitious_statistic_prior, title=title, xlabel=xlabel)
#     figure_utilities.plot_histogram(ax, fictitious_statistic_posterior, title=title, xlabel=xlabel, color = "red")

axes[1].set_ylabel("")
axes[2].set_ylabel("")
# plt.legend(["Prior", "Posterior", "True Statistic"])
plt.legend(["Prior", "True Statistic"])

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_FIGURES, "prior_predictive_analysis.png"))
plt.show()
plt.close(fig)

## 2b. Evidence of Implementation Correctness

In this section, we perform Geweke's joint distribution test for correct implementation of our Gibbs sampler (Geweke 2004). We use the following steps:
1. Simulate unobservables from their priors. Note that in this step, it is not necessary for us to choose "sensible" priors, as this test is for implementation correctenss and is not intended to answer any research questions. We thus choose extremely simple priors in this section only.
2. Simulate a sample of y_is conditional on these unobservables
3. Simulate a sample of unobservables from their conditional distribution using our Gibbs sampler
4. Repeat steps 2 and 3 1000 times. Now, we have 1000 draws of our unobservables.
5. Compare the distribution of these unobservables to our prior distributions.

In [ ]:
# Temporarily set the prior parameters on our betas to something simple.
simple_prior_parameters = prior_parameters.copy()
simple_prior_parameters['beta_underbar'] = np.zeros(k)
simple_prior_parameters['H_underbar'] = np.eye(k)
simple_mixture_sampler = MixtureSampler(generator, simple_prior_parameters, X, y)

current_draw_of_betas, current_draw_of_h, current_draw_of_alpha_js, current_draw_of_h_js, current_draw_of_mu_js = simple_mixture_sampler.sample_from_prior()
current_draws_of_s_is = generator.choice(prior_parameters["m"], size=len(y), p=current_draw_of_alpha_js)

all_betas_prior = np.vstack(list(simple_mixture_sampler.sample_from_prior()[0] for i in range(N)))
all_betas_posterior = []
# 
for i in range(N):
    # Draw y_is
    current_means_for_each_observation =  current_draw_of_mu_js[current_draws_of_s_is] + (current_draw_of_betas @ X.T)
    current_h_js_for_each_observation = current_draw_of_h_js[current_draws_of_s_is]
    current_y_is = generator.multivariate_normal(np.ravel(current_means_for_each_observation), np.diag(1 / current_h_js_for_each_observation))

    # Assign simulated y_is to the sampler.
    mixture_sampler.y = current_y_is

    # Draw new unobservables conditional on y_is
    current_draw_of_betas, current_draw_of_h, current_draw_of_alpha_js, current_draw_of_h_js, current_draw_of_mu_js, current_draws_of_s_is = simple_mixture_sampler.sample_from_posterior()

    all_betas_posterior.append(current_draw_of_betas)
all_betas_posterior = np.vstack(all_betas_posterior)


# Restore beta prior parameters to original values.
prior_parameters['beta_underbar'] = (np.linalg.inv(X.T @ X) @ (X.T @ y))
prior_parameters['H_underbar'] = X.T @ X

In [ ]:
# Test for equality of prior beta samples and simulated posterior beta samples (first moment).

geweke_test_results_first_moment = pd.DataFrame(np.column_stack([np.mean(all_betas_prior, axis=0),
                                    np.mean(all_betas_posterior, axis=0),
                                    stats.ttest_ind(all_betas_prior, all_betas_posterior, axis=0).pvalue]),
                                   columns=["Samples from $N(\\underline{0}, I_k)$",
                                            "Posterior Samples of $\\beta$",
                                            "P-Value From Difference in Means Test"],
                                   index=["$\\beta_{0}$", "$\\beta_{tchratio}$", "$\\beta_{pctel}$", "$\\beta_{percap}$", "$\\beta_{lnch_pct}$"])
# Export to LaTeX.
filename = os.path.join(OUTPUT_TABLES, "geweke_joint_distribution_test_first_moment.tex")
latex = (geweke_test_results_first_moment
         .style
         .format(formatter="{:,.2f}")
         .to_latex(None,
                   column_format="lccc",
                   hrules=True,
                   clines="skip-last;data")).replace("{*}", "{4cm}")
with open(filename, 'w') as file:
    file.write(latex)
geweke_test_results_first_moment

In [ ]:
# Plot the distribution of first moments of each beta
fig, axes = plt.subplots(1, 5, figsize=(7.5, 5), sharey=True)
fig.suptitle("Joint distribution test for individual components of $\\beta$ (first moment)")
titles = ["$\\beta_{0}$", "$\\beta_{tchratio}$", "$\\beta_{pctel}$", "$\\beta_{percap}$", "$\\beta_{lnch_{pct}}$"]

# axes = [ax for l in axes for ax in l]

for ax, beta_index, title in zip(axes, range(all_betas_prior.shape[1]), titles):
    current_beta_prior = pd.Series(np.reshape(all_betas_prior[:, beta_index], -1))
    current_beta_posterior = pd.Series(np.reshape(all_betas_posterior[:, beta_index], -1))
    
    current_beta_prior.plot(ax = ax, kind='kde', alpha = 0.5, color = 'blue')
    current_beta_posterior.plot(ax = ax, kind='kde', alpha = 0.5, color = 'red')
    ax.title.set_text(title)
    
plt.legend(["Prior", "Posterior"], bbox_to_anchor=(1.05, 1.0), loc='upper left')
axes[0].set_ylabel("")
axes[1].set_ylabel("")
axes[2].set_ylabel("")
axes[3].set_ylabel("")
axes[4].set_ylabel("")

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_FIGURES, "implementation_check_betas_first_moment.png"))
plt.show()
# plt.close(fig)

In [ ]:
# Test for equality of prior beta samples and simulated posterior beta samples (second moment).

geweke_test_results_second_moment = pd.DataFrame(np.column_stack([np.mean(np.square(all_betas_prior), axis=0),
                                    np.mean(np.square(all_betas_posterior), axis=0),
                                    stats.ttest_ind(np.square(all_betas_prior), np.square(all_betas_posterior), axis=0).pvalue]),
                                   columns=["Samples from $N(\\underline{0}, I_k)$",
                                            "Posterior Samples of $\\beta$",
                                            "P-Value From Difference in Second Moments Test"],
                                   index=["$\\beta_{0}$", "$\\beta_{tchratio}$", "$\\beta_{pctel}$", "$\\beta_{percap}$", "$\\beta_{lnch_pct}$"])
# Export to LaTeX.
filename = os.path.join(OUTPUT_TABLES, "geweke_joint_distribution_test_second_moment.tex")
latex = (geweke_test_results_first_moment
         .style
         .format(formatter="{:,.2f}")
         .to_latex(None,
                   column_format="lccc",
                   hrules=True,
                   clines="skip-last;data")).replace("{*}", "{4cm}")
with open(filename, 'w') as file:
    file.write(latex)
geweke_test_results_second_moment

In [ ]:
# Plot the distribution of second moments of each beta
fig, axes = plt.subplots(1, 5, figsize=(7.5, 5), sharey=True)
fig.suptitle("Joint distribution test for individual components of $\\beta$ (second moment)")
titles = ["$\\beta_{0}$", "$\\beta_{tchratio}$", "$\\beta_{pctel}$", "$\\beta_{percap}$", "$\\beta_{lnch_{pct}}$"]

# axes = [ax for l in axes for ax in l]

for ax, beta_index, title in zip(axes, range(all_betas_prior.shape[1]), titles):
    current_beta_prior = pd.Series(np.reshape(all_betas_prior[:, beta_index], -1))**2
    current_beta_posterior = pd.Series(np.reshape(all_betas_posterior[:, beta_index], -1))**2
    
    current_beta_prior.plot(ax = ax, kind='kde', alpha = 0.5, color = 'blue')
    current_beta_posterior.plot(ax = ax, kind='kde', alpha = 0.5, color = 'red')
    ax.title.set_text(title)
    
plt.legend(["Prior", "Posterior"], bbox_to_anchor=(1.05, 1.0), loc='upper left')
axes[0].set_ylabel("")
axes[1].set_ylabel("")
axes[2].set_ylabel("")
axes[3].set_ylabel("")

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_FIGURES, "implementation_check_betas_second_moment.png"))
plt.show()
plt.close(fig)

## 2c. Evidence of Posterior Simulation Convergence

In this section, we confirm that our posterior simulator has converged. We show that our posterior simulator has converged using two main methods. Here are the steps we take to show convergence using method 1:
1. Draw a sample from the joint posterior of the unobservables using our Gibbs sampler.
2. Draw a fictitious sample of y_is from the likelihood
3. Repeat many times
4. Now, we have many samples of y_is. Compute the mean and standard deviation of each sample and plot.

In [ ]:
# Increase N to check for convergence
# Note: N = 10,000 took about 6 minutes to run.
N = 10000
draws_of_betas, draws_of_h, draws_of_alpha_js, draws_of_h_js, draws_of_mu_js, draws_of_s_is = mixture_sampler.sample_from_posterior(N)


all_simulated_y_is = []
for draw_of_beta, draw_of_h, draw_of_alpha_js, draw_of_h_js, draw_of_mu_js, draw_of_s_is in zip(draws_of_betas, draws_of_h, draws_of_alpha_js, draws_of_h_js, draws_of_mu_js, draws_of_s_is):

    # Draw y_is
    current_means_for_each_observation =  draw_of_mu_js[draw_of_s_is] + (draw_of_beta @ X.T)
    current_h_js_for_each_observation = draw_of_h_js[draw_of_s_is]
    current_y_is = generator.multivariate_normal(np.ravel(current_means_for_each_observation), np.diag(1 / current_h_js_for_each_observation))

    all_simulated_y_is.append(current_y_is)
all_simulated_y_is = np.vstack(all_simulated_y_is)
means = np.mean(all_simulated_y_is, axis=1)

fig, ax = plt.subplots(1, 1)
ax.plot(means)
ax.set_title("Means of Simulated $Y_i$'s Drawn Conditional on Gibbs-Sampled Unobservables")
ax.set_ylabel("$\\bar{y} = \\frac{1}{n}\sum_{i=1}^{n}y_i$")
ax.set_xlabel("Algorithm Iteration")

plt.savefig(os.path.join(OUTPUT_FIGURES, "convergence_check_1.png"))
# plt.close(fig)

plt.show()

# Re-set N to be as above
N = 1000

In our second method, we compare the distribution of the second quarter of draws to the distribution of the fourth quarter of draws. We do this for every unobservable. Multidimensional unobservables (betas, h_j's, mu_j's) are averaged to avoid "labeling" issues; we weight these averages using the alpha_j's, following Professor Norets' recommendation. For each unobservable, we take the following steps:
1. Plot the second 25 percent of values and the last 25 percent of values.
2. Perform a difference in means test between the second 25 percent of draws and the last 25 percent of draws.

In [ ]:
# Draw from posterior. NOTE: May be able to drop this check; Prof. Norets said first one is better anyway
# NOTE: DO NOT NEED TO RUN AGAIN, N = 100,000 TOOK A LONG TIME >30 MIN, MAYBE CLOSE TO AN HOUR TO RUN
N = 10000
draws_of_betas, draws_of_h, draws_of_alpha_js, draws_of_h_js, draws_of_mu_js, draws_of_s_is = mixture_sampler.sample_from_posterior(num_samples=100000)
draws_of_betas = np.mean(draws_of_betas, axis=1)
draws_of_h_js = np.average(draws_of_h_js, axis=1, weights=draws_of_alpha_js)
draws_of_mu_js = np.average(draws_of_mu_js, axis=1, weights=draws_of_alpha_js)

unobservables_to_check = [draws_of_betas, draws_of_h_js, draws_of_mu_js, draws_of_h, draws_of_s_is, draws_of_alpha_js]
fig, axes = plt.subplots(2, 3, figsize=(10, 5))
fig.suptitle("Comparison of Second Quarter of Draws to Fourth Quarter of Draws")
axes = [ax for l in axes for ax in l]
titles = ["$\\beta$\'s", "$h_j$\'s", "$\\mu_j$\'s", "$h$\'s", "$s_i$\'s", "$\\alpha_j$\'s"]
xlabels = ["$\\bar{\\beta}$", "$\\sum_{j=1}^{3} h_j*\\alpha_j$", "$\\sum_{j=1}^{3} \\mu_j*\\alpha_j$", "$h$", "$\\bar{s_i}$", "$\\bar{\\alpha_j}$"]
for ax, unobservable, xlabel, title in zip(axes, unobservables_to_check, xlabels, titles):
    current_unobservable = pd.Series(np.reshape(unobservable, -1))
    current_unobservable.loc[0.25*N:0.5*N+1].plot(ax=ax, kind='kde', color='blue', alpha = 0.5)
    current_unobservable.loc[0.75*N:N].plot(ax=ax, kind='kde', color='red', alpha = 0.5)

    p_value = stats.ttest_ind(current_unobservable.loc[0.25*N:0.5*N+1],current_unobservable.loc[0.75*N:N] ).pvalue
    ax.text(0.05, 0.95, f"p-value:\n{round(p_value, 2)}", horizontalalignment='left', verticalalignment='top', transform=ax.transAxes, fontsize=6)
    ax.set_xlabel(xlabel)
    ax.set_ylabel("")
    ax.set_title(title)
plt.legend([Line2D([0], [0], color='blue', lw=4), Line2D([0], [0], color='red', lw=4)], ['Second Quarter of Draws', 'Fourth Quarter of Draws'], bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_FIGURES, "convergence_check_2.png"))
plt.show()
plt.close(fig)


## 2d. Comparison of Models

In this section, we compare model fit across different levels of model complexity. The model previously explored in this analysis used three mixture components. We next estimate two other models, with 5 and 7 mixture components respectively, and compare their model fit to our original model

Michael: second cell here throws error when writing to Latex bc model_comparison_table is a series and doesn't have attribute 'style'. Otherwise log likelihoods look good/okay?

In [ ]:
N = 10000
logscores = {2: [], 3: [], 5: [], 7: []}
for m in [2, 3, 5, 7]:
    current_prior_parameters = prior_parameters.copy()
    current_prior_parameters["m"] = m
    current_prior_parameters['alpha_underbar'] = np.ones(current_prior_parameters['m'])
    for split in range(50):
        # Split into estimation and evaluation samples.
        train_df, test_df = train_test_split(df, test_size=0.15)
        current_train_X = train_df[X_variables].values
        current_train_y = train_df[y_variable].values
        current_test_X = test_df[X_variables].values
        current_test_y = test_df[y_variable].values

        current_sampler = MixtureSampler(generator, current_prior_parameters, current_train_X, current_train_y)

        # Draw unobservables.
        [draws_of_betas, draws_of_h, draws_of_alpha_js, draws_of_h_js, draws_of_mu_js, draws_of_s_is] = current_sampler.sample_from_posterior(num_samples=N)

        # Save last draw.
        current_beta = draws_of_betas[-1]
        current_h = draws_of_h[-1]
        current_alpha_js = draws_of_alpha_js[-1]
        current_h_js = draws_of_h_js[-1]
        current_mu_js = draws_of_mu_js[-1]
        current_s_i_equals_js = np.random.choice(m, p=current_alpha_js, size=len(current_test_y))

        current_logscore = 0
        for i in range(len(current_test_y)):
            current_component = current_s_i_equals_js[i]
            current_mean = current_mu_js[current_component] + (current_beta @ current_test_X[i].T)
            current_precision = current_h_js[current_component]
            current_logscore += stats.norm(current_mean, 1 / current_precision).pdf(current_test_y[i])
        logscores[m].append(current_logscore)
        
N = 1000

In [ ]:
mean_logscores = []
for component in logscores.keys():
    mean_logscore = sum(logscores[component]) / len(logscores[component])
    mean_logscores.append(mean_logscore)
model_comparison_table = pd.DataFrame(mean_logscores, index=["2", "3", "5", "7"], columns = ["Mean Logscore"])
model_comparison_table.index.name = "Number of Mixture Components"

# Export to LaTeX.
filename = os.path.join(OUTPUT_TABLES, "log_scores.tex")
latex = (model_comparison_table
         .style
         .format(formatter="{:,.2f}")
         .to_latex(None,
                   hrules=True,
                   clines="skip-last;data")).replace("{*}", "{4cm}")
with open(filename, 'w') as file:
    file.write(latex)
model_comparison_table

# 2e. Prior Sensitivity Analysis
It looks like our posterior simulations are very sensitive to prior hyperparameter values even when the number of simulations is large (n = 500,000).

In [ ]:
strong_prior_parameters = prior_parameters.copy()
weak_prior_parameters = prior_parameters.copy()

N = 10000

unchanged_prior_parameters_list = ['m', 'mu_j_underbar', 'beta_underbar', 's2_j_underbar', 's2_underbar', 'alpha_underbar']

for key in prior_parameters:
    if key not in unchanged_prior_parameters_list:
        strong_prior_parameters[key] = strong_prior_parameters[key] * 5
        weak_prior_parameters[key] = weak_prior_parameters[key] * 1/5

# Set up new samplers. "Original" mixture sampler is called mixture_sampler
strong_mixture_sampler = MixtureSampler(generator, strong_prior_parameters, X, y)
weak_mixture_sampler = MixtureSampler(generator, weak_prior_parameters, X, y)

# Draw from posteriors
strong_betas, strong_h, strong_alpha_js, strong_h_js, strong_mu_js, strong_s_is = strong_mixture_sampler.sample_from_posterior(N)
weak_betas, weak_h, weak_alpha_js, weak_h_js, weak_mu_js, weak_s_is = weak_mixture_sampler.sample_from_posterior(N)
original_betas, original_h, original_alpha_js, original_h_js, original_mu_js, original_s_is = mixture_sampler.sample_from_posterior(N)

N = 1000


In [ ]:
least_squares_betas = np.linalg.inv(X.T@X) @ (X.T @ y)
prior_sensitivity = pd.DataFrame(np.column_stack([least_squares_betas, weak_betas.mean(axis = 0), original_betas.mean(axis = 0), strong_betas.mean(axis = 0)]), 
                                 columns = ["Least Squares/Prior", "Weak Prior", "Original Prior", "Strong Prior"], 
                                 index=["$\\beta_0$", "$\\beta_{tchratio}$", "$\\beta_{pctel}$", "$\\beta_{percap}$", "$\\beta_{lnch_{pct}}$"])
# Export to LaTeX.
filename = os.path.join(OUTPUT_TABLES, "prior_sensitivity.tex")
latex = (prior_sensitivity
         .style
         .format(formatter="{:,.2f}")
         .to_latex(None,
                   column_format="lccccc",
                   hrules=True,
                   clines="skip-last;data")).replace("{*}", "{4cm}")
with open(filename, 'w') as file:
    file.write(latex)
prior_sensitivity

# 3a. Estimation results
Should provide coefficients for least squares estimate; posterior using 'weak', 'original', 'strong' priors; prior means for each of m = 2, 3, 5, 7

In [ ]:
N = 10000
original_prior_parameters = prior_parameters.copy()

strong_betas_by_m = []
weak_betas_by_m = []
original_betas_by_m = []

# for m in [2, 3, 5, 7]:
for m in [3]:
    strong_prior_parameters["m"] = m
    weak_prior_parameters["m"] = m
    original_prior_parameters["m"] = m
    
    strong_prior_parameters['alpha_underbar'] = np.ones(strong_prior_parameters['m'])
    weak_prior_parameters['alpha_underbar'] = np.ones(weak_prior_parameters['m'])
    original_prior_parameters['alpha_underbar'] = np.ones(original_prior_parameters['m'])
    
    # Make generators
    strong_mixture_sampler = MixtureSampler(generator, strong_prior_parameters, X, y)
    weak_mixture_sampler = MixtureSampler(generator, weak_prior_parameters, X, y)
    original_mixture_sampler = MixtureSampler(generator, original_prior_parameters, X, y)

    # Draw unobservables.
    strong_betas, strong_h, strong_alpha_js, strong_h_js, strong_mu_js, strong_s_is = strong_mixture_sampler.sample_from_posterior(N)
    weak_betas, weak_h, weak_alpha_js, weak_h_js, weak_mu_js, weak_s_is = weak_mixture_sampler.sample_from_posterior(N)
    original_betas, original_h, original_alpha_js, original_h_js, original_mu_js, original_s_is = mixture_sampler.sample_from_posterior(N)
    
    if m == 3: original_betas_stored = original_betas.copy()
    
    # Store beta values
    strong_betas_by_m.append(strong_betas.mean(axis = 0))
    weak_betas_by_m.append(weak_betas.mean(axis = 0))
    original_betas_by_m.append(original_betas.mean(axis = 0))
    
strong_betas_by_m = np.vstack(strong_betas_by_m)
weak_betas_by_m = np.vstack(weak_betas_by_m)
original_betas_by_m = np.vstack(original_betas_by_m)

In [ ]:
estimation_results_column_names = ["Weak, m = 2", "Weak, m = 3", "Weak, m = 5", "Weak, m = 7", "Original, m = 2", "Original, m = 3", "Original, m = 5", "Original, m = 7", "Strong, m = 2", "Strong, m = 3", "Strong, m = 5", "Strong, m = 7"]
estimation_results_table = pd.DataFrame(np.column_stack([weak_betas_by_m.T, original_betas_by_m.T, weak_betas_by_m.T]), 
                   columns = estimation_results_column_names, 
                   index=["$\\beta_0$", "$\\beta_{tchratio}$", "$\\beta_{pctel}$", "$\\beta_{percap}$", "$\\beta_{lnch_{pct}}$"])

# Export to LaTeX.
# Fix by hand in the report
filename = os.path.join(OUTPUT_TABLES, "estimation_results.tex")
latex = (estimation_results_table
         .style
         .format(formatter="{:,.2f}")
         .to_latex(None,
                   column_format="lcccccccccccccccc",
                   hrules=True,
                   clines="skip-last;data")).replace("{*}", "{4cm}")
with open(filename, 'w') as file:
    file.write(latex)
estimation_results_table

In [ ]:
beta_tchratio = pd.Series(np.reshape(original_betas_stored[:, 1], -1))

fig, axes = plt.subplots(1, 2, figsize = (10, 5))

fig.suptitle("Estimation Results of $\\beta_{tchratio}$")
axes[0].plot(beta_tchratio)
axes[0].set_xlabel("Iteration")
axes[0].set_ylabel("$\\beta_{tchratio}$")

beta_tchratio.plot(ax = axes[1], kind = 'kde', color = 'red')
axes[1].set_xlabel("$\\beta_{tchratio}$")

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_FIGURES, "estimation_results_figure.png"))
plt.show()
plt.close(fig)

# 3b. Decision problem

In [ ]:
cd_ratio_list = [150, 200, 250]
beta_tchratio_list = [estimation_results_table.iloc[1, 2], estimation_results_table.iloc[1, 6], estimation_results_table.iloc[1, 10]]
print(beta_tchratio_list)

bayes_action_array = np.array([np.sqrt((-1 * cd_ratio* (1/beta_tchratio))) for cd_ratio in cd_ratio_list for beta_tchratio in beta_tchratio_list ]).reshape((len(cd_ratio_list), len(beta_tchratio_list)))



bayes_aciton_df = pd.DataFrame(np.column_stack(bayes_action_array),
                               columns = ["150", "200", "250"],
                               index = ["Weak Prior", "Original Prior", "Strong Prior"])

print(bayes_aciton_df)
# Export to Latex
# Modified slightly by hand
filename = os.path.join(OUTPUT_TABLES, "bayes_action_table.tex")
latex = (bayes_aciton_df
         .style
         .format(formatter="{:,.2f}")
         .to_latex(None,
                   column_format="lccc",
                   hrules=True,
                   clines="skip-last;data")).replace("{*}", "{4cm}")
with open(filename, 'w') as file:
    file.write(latex)